In [ ]:
from dolfin import *

%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from petsc4py import PETSc

from gmsfem.utils import *
# from gmsfem.numpy.p1case import *    # one parameter
from gmsfem.petsc.p1case import *  # many parameters
from gmsfem.subdiv import *
from autoload.optimal_distribution import *

In [ ]:
# %load_ext line_profiler
# %lprun -f func_name func_name(*args)

In [ ]:
def get_coord(reg_id):
    col = reg_id % (n_blocks-1)
    row = reg_id // (n_blocks-1)
    return col, row
    
def build_V(reg_id):
    col, row = get_coord(reg_id)
    mesh = UnitSquareMesh(2*n_el, 2*n_el)
    mesh.translate(Point(col*.5, row*.5))
    mesh.scale(2./n_blocks)

    return FunctionSpace(mesh, 'P', 1)

def grad_sqr_norm(df):
    u, v = df 
    u, v = map(project, [u, v])
    return u.vector()[:]**2 + v.vector()[:]**2

# P1 Space 

## One Parameter

In [ ]:
n_el = 16
n_blocks = 4 # must be > 3
N_el = n_el * n_blocks
N_cnbh = (n_blocks-1)**2

M_off = 50
M_on = 30
eta = 1e3
# -----------------

K = get_simple_kappa(eta, N_el, rho=20, seed=123)
W = K.function_space()
fine_mesh = W.mesh()

RHS = Expression('sin(2*pi*x[0])*sin(2*pi*x[1])', pi=pi, degree=1)
#RHS = Expression('exp(-pow(x[0]-.5, 2)-pow(x[1]-.5, 2))', pi=np.pi, degree=1)
pairs = overlap_map(n_blocks-1)

cores = [None] * (n_blocks-1)*(n_blocks-1)
K_list, RHS_list = [], []

Nv_ms = np.empty((len(cores), M_off, (2*n_el+1)**2))
ms_dofs = np.empty((len(cores), M_off, ((N_el+1)**2)))

In [ ]:
%%time
for i in range(len(cores)):
    V = build_V(i)
    kappa = project(K, V)
    K_list.append(kappa)
    RHS_list.append(project(RHS, V))

    cores[i] = GMsFEUnit(i, n_el, n_blocks)
    Nv = cores[i].snapshotSpace(kappa)
    
    Nv,w = cores[i].modelReduction(kappa, Nv, M_off)
    Nv_ms[i] = cores[i].multiscaleDOFs(kappa, Nv)
    ms_dofs[i] = zero_extrapolate(Nv_ms[i], V, W, *get_coord(i))

clear_cache()

In [ ]:
%%time
A = np.zeros((len(cores)*M_off, len(cores)*M_off))
b = np.zeros(len(cores)*M_off)

for i in range(len(cores)):
    A_ii, b_i = cores[i].diagonalBlock(K_list[i], Nv_ms[i], RHS_list[i])
    A[i*M_off:(i+1)*M_off, i*M_off:(i+1)*M_off] = A_ii
    b[i*M_off:(i+1)*M_off] = b_i
    
for i,j in pairs:
    A_ij = cores[i].offdiagonalBlock(K_list[i], Nv_ms[i], Nv_ms[j], j-i)
    A[i*M_off:(i+1)*M_off, j*M_off:(j+1)*M_off] = A_ij

In [ ]:
sigma = 0 #1e-9
i_lower = np.tril_indices(len(A), -1)
A[i_lower] = A.T[i_lower]
A += sigma*np.identity(len(A))

sol = Function(W)
u = scipy.linalg.solve(A, b, assume_a='pos')
sol.vector().set_local(u @ ms_dofs.reshape(-1, W.dim()))

In [ ]:
%%time
v1 = TrialFunction(W)
v2 = TestFunction(W)
dx = Measure('dx', W)
a = K*dot(grad(v1), grad(v2)) * dx
L = RHS*v2*dx
u_f = Function(W)
bc = DirichletBC(W, Constant(0.), lambda x,on : on)

solve(a==L, u_f, bc)

In [ ]:
fig = plt.figure(figsize=(29, 24))
ax1 = plt.subplot(221)
plot(K)
ax2 = plt.subplot(222)
p1 = plot(RHS, mesh=fine_mesh)

ax3 = plt.subplot(223)
plot(sol);
#ax3.set_title('GMsFESolver', size=32)

ax4 = plt.subplot(224)
p2 = plot(u_f);
#ax4.set_title('FEM', size=32)

cax1 = fig.colorbar(p1, ax=[ax1, ax2], location='right', shrink=.75, use_gridspec=True)
cax2 = fig.colorbar(p2, ax=[ax3, ax4], location='right', shrink=.75, use_gridspec=True)
cax1.ax.tick_params(labelsize=16)
cax2.ax.tick_params(labelsize=16)
ax3.text(
    .5, -.08, f'L2 distance between them: {norm(project(u_f - sol, W)):.3}',
    transform=ax3.transAxes, alpha=.5, color='r', size=22)

#plt.tight_layout()
#plt.savefig(f'../figures/np4sin_el{n_el}bl{n_blocks}off{M_off}on{M_on}.png')

## Many parameters

In [ ]:
n_el = 32
n_blocks = 4

N_el = n_el * n_blocks
N_cnbh = (n_blocks-1)**2

M_off = 30
M_on = 15
eta = [1e2]# 3e2]# 4e2]
rho = N_el // 3
# -----------------

K = get_simple_kappa(eta, N_el, rho=rho, seed=123)
W = K[0].function_space()
fine_mesh = W.mesh()

pairs = overlap_map(n_blocks-1)
cores = [None] * (n_blocks-1)*(n_blocks-1)

avg_K = Function(W)
for kappa in K:
    avg_K.vector().axpy(1/3, kappa.vector())

V_set, avgK_set = [], []
for i in range(len(cores)):
    V_set.append(build_V(i))
    avgK_set.append(project(avg_K, V_set[i]))
    cores[i] = GMsFEUnit(i, n_el, n_blocks)
clear_cache()
    
K_set = np.empty((len(eta), len(cores)), 'O')
for k, gkappa in enumerate(K):
    for i in range(len(cores)):
        lkappa = project(gkappa, V_set[i])
        K_set[k, i] = lkappa

Nv_off = np.empty((len(cores), M_off, (2*n_el+1)**2))
Nv_ms = np.empty((len(cores), M_on, (2*n_el+1)**2))
ms_dofs = np.empty((len(cores), M_on, ((N_el+1)**2)))

In [ ]:
%%time
eps1 = 1e-12
change_form = False

if change_form:
    acc = np.zeros((2*n_el+1)**2)
    for i, kappa in enumerate(avgK_set):
        xi = cores[i].partitionFunction(kappa)
        acc += grad_sqr_norm(grad(xi))

for i in range(len(cores)):
    if change_form:
        km = Function(V_set[i])
        km.vector()[:] = avgK_set[i].vector().get_local()
        km.vector()[:] *= acc
    
    Nv = np.empty((len(eta), 8*n_el, (2*n_el+1)**2))
    for k, kappa in enumerate(K_set):
        Nv[k] = cores[i].snapshotSpace(kappa[i])
    Nv = Nv.reshape(len(eta)*8*n_el, -1)
    
    if not change_form: km = None
    Nv_off[i], w_off = cores[i].modelReduction(avgK_set[i], Nv, M_off, eps=eps1, km=km)

In [ ]:
%%time
eps2 = 1e-14
change_form = False

perm_id = 0
Kt = K_set[perm_id]

if change_form:
    acc = np.zeros((2*n_el+1)**2)
    for i, kappa in enumerate(Kt):
        xi = cores[i].partitionFunction(kappa)
        acc += grad_sqr_norm(grad(xi))

for i in range(len(cores)):
    if change_form:
        km = Function(V_set[i])
        km.vector()[:] = Kt[i].vector().get_local()
        km.vector()[:] *= acc

    if not change_form: km = None
    Nv_on, w_on = cores[i].modelReduction(Kt[i], Nv_off[i], M_on, eps=eps2, km=km)
    Nv_ms[i] = cores[i].multiscaleDOFs(Kt[i], Nv_on)
    ms_dofs[i] = zero_extrapolate(Nv_ms[i], V_set[i], W, *get_coord(i))

In [ ]:
RHS = Expression('sin(2*pi*x[0])*sin(2*pi*x[1])', pi=np.pi, degree=1)
#RHS = Expression('exp(-pow(x[0]-.5, 2)-pow(x[1]-.5, 2))', pi=pi, degree=1)

RHS_set = []
for i in range(len(cores)):
    RHS_set.append(project(RHS, V_set[i]))

In [ ]:
%%time
bc = len(cores)*M_on
A = PETSc.Mat()
A.createAIJ((bc, bc))
A.setUp()

b = PETSc.Vec()
b.createSeq(bc)

for i in range(len(cores)):
    A_ii, b_i = cores[i].diagonalBlock(Kt[i], Nv_ms[i], RHS_set[i])
    I = np.arange(i*M_on, (i+1)*M_on, dtype='i4')
    A.setValues(I, I, A_ii.getDenseArray().flatten())
    b.setValues(I, b_i)
    
for i,j in pairs:
    A_ij = cores[i].offdiagonalBlock(Kt[i], Nv_ms[i], Nv_ms[j], j-i)
    I = np.arange(i*M_on, (i+1)*M_on, dtype='i4')
    J = np.arange(j*M_on, (j+1)*M_on, dtype='i4')
    A.setValues(I, J, A_ij.getDenseArray().flatten())
    A.setValues(J, I, A_ij.getDenseArray().T.flatten())

A.assemble()
A = PETScMatrix(A)
b = PETScVector(b)

In [ ]:
%%time
solver = LUSolver(A, 'superlu_dist')
solver.parameters['symmetric'] = True

u = Vector(MPI.comm_self, len(b))
solver.solve(u, b)

sol = Function(W)
sol.vector().set_local(u @ ms_dofs.reshape(-1, W.dim()))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.plot(w_off/w_off[0])
ax1.set_yscale('log')
ax1.set_ylabel(r'$\frac{\lambda_i}{\lambda_0}$', size=22)
ax1.set_title('Offline', size=22)

ax2.plot(w_on/w_on[0])
ax2.set_yscale('log')
ax2.set_title('Online', size=22)
ax2.set_xlabel('i', size=22);

In [ ]:
%%time
v1 = TrialFunction(W)
v2 = TestFunction(W)
dx = Measure('dx', W)
perm_id = 0
a = K[perm_id]*dot(grad(v1), grad(v2)) * dx
L = RHS*v2*dx
u_f = Function(W)
bc = DirichletBC(W, Constant(0.), lambda x,on: on)

solve(a==L, u_f, bc)

In [ ]:
fig = plt.figure(figsize=(9, 27))
ax0 = plt.subplot(311)
plot(K[perm_id])
ax1 = plt.subplot(312)
plot(sol);
#ax1.set_title('GMsFESolver', size=32)

ax2 = plt.subplot(313)
p = plot(u_f);
#ax2.set_title('FEM', size=32)

#cax = fig.colorbar(p, ax=[ax1, ax2], shrink=.75)
#cax.ax.tick_params(labelsize=16)
ax1.text(
    -.05, -.1, f'L2 distance between them: {norm(project(u_f - sol, W)):.3}',
    transform=ax1.transAxes, alpha=.5, color='r', size=22)
#plt.savefig(f'../figures/r{perm_id}el{n_el}bl{n_blocks}off{M_off}on{M_on}eps{(eps1+eps2)/2}sig{sigma}.png')

# Colored Partition

In [ ]:
cp = ColoredPartition(9, 4)

In [ ]:
cp.partition(9)

In [ ]:
print((cp.map['c'] != None).sum())
print(cp.map['c'])
print()
print(cp.map['r'])

# Python CPP module

In [ ]:
with open('cutils/pymodule3.cpp', 'r') as fp:
    cpp_code = fp.read()

In [ ]:
cutils = compile_cpp_code(cpp_code, include_dirs=['cutils'])